In [1]:
import pandas as pd

In [9]:
events = pd.read_excel("events.xlsx", engine='openpyxl')
key_segments = pd.read_excel("key segments.xlsx", engine='openpyxl')
segment_data = pd.read_csv("segment_data.csv", sep=";")
monday_speeds = pd.read_csv("monday-speeds.csv", sep=";")
segments_ids_nord_direction = pd.read_csv("north_direction_road_segments.txt", header=None, names=['SegmentId'])

### Event Document

In [10]:
events.head(1)

,Skapad_År,Skapad_Månad,NAME,EVENT_CODE,EVENT_TEXT,Location_text,Väg,Riktning,LANE_MASK,CREATION_TIME,COMPLETION_TIME,START_TIME,END_TIME,Note,X,Y
0,2019,Januari,2019-01-01/03:34:40-1/4,35,Olycka,E4 från Trafikplats Sörljusne (204) till Trafi...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-01 03:34:40,2019-01-01 06:01:30,2019-01-01 03:34:40,2019-01-01 06:15:00,I höjd med Böles-Noran i södergående. En perso...,6774947,606946


### SegmentID direction and coordinates

In [11]:
key_segments.head(1)

,RouteID,SegmentID,ValidFrom,ValidTo,Längd,Färdriktn,Latitud,Longitud,Ordning,Väg,Antal kf,Hastighet,Korsning,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1520100057,1071754891,2019-04-16,9999-12-31 00:00:00,930,Syd,"61,165335","16,851284",1,83,1kf,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### SegmentId meta data

In [12]:
segment_data.head(1)

,SegmentId,Length,Road,Antal kf,Max_speed
0,1071754891,930,83,1kf,90


### SegmentID and speeds

In [13]:
monday_speeds.head(1)

,SegmentId,StartTime,EndTime,Speed
0,1071707139,2019-06-03 00:00:00,2019-06-03 00:30:00,NaN


# The Goal

- Create a deep learning model to predict traffic speed after a traffic event until it returns to normal speed

# What variables needed to do predictions?
- SegmentID
- Speed in time
- Events
    - Code/Text
    - CREATION_TIME 	
    - COMPLETION_TIME 	
    - START_TIME 	
    - END_TIME

- Weather data is not needed to do the project and therefore not included here. Most important is to link speed and events first and then we can expand with weather data

# Observations

#### Event Document
- The event code
- X and Y location (this is most likely not latitude and longitude)

#### SegmentID and Speed
- segmentID
- Speed

#### SegmentId and coordinates
- SegmentID
- Latitude and Longitude coordinates

# My take on it

It's unclear if Latitude and Longitude can be connected to the X,Y from the event document which is required to link SegmentID, Speed and Events.

<br>

Therefore, questions to answer
- What type of coordinate do the X,Y represent in the event document since they don't look like latitude and longitude?

In [14]:
df = pd.read_csv("../data/events-with-lat-long-segmentID.csv", sep=",")

In [15]:
this_segment = df[df['SegmentID'] == 226255131]

In [16]:
this_segment

,Unnamed: 0,Skapad_År,Skapad_Månad,NAME,EVENT_CODE,EVENT_TEXT,Location_text,Väg,Riktning,LANE_MASK,CREATION_TIME,COMPLETION_TIME,START_TIME,END_TIME,Note,X,Y,latitude,longitude,SegmentID
1338,1338,2021,Juli,2021-07-27/23:16:50-1/4,40,Vägarbete,E4 från Axmartavlan (202) till Trafikplats Tön...,E4,Norr,OOOOOOOOO|XOOOOOOOO,2021-07-27 23:16:50,2021-07-28 00:32:15,2021-07-27 23:16:50,2021-07-28 00:30:00,Underhållsarbete i höjd med Högbacka. Risk för...,6765123,607357,61.00636,16.985545,226255131


In [39]:
segment_ids = []
segment_ids_count = []

#temp = df.loc[(df['EVENT_CODE'] != 40) & df['EVENT_CODE'] != 2)] 

temp = df.loc[(df['EVENT_CODE'] != 40) & 
              (df['EVENT_CODE'] != 2)] 


for segment_id in temp['SegmentID'].values:
    
    
    counts = temp['SegmentID'].value_counts()
    
    #print(counts)
    
    if segment_id not in segment_ids:
        
        
        count = counts.get(segment_id)
        
        print(segment_id, count)
        
        segment_ids_count = [str(segment_id), count]
        segment_ids.append(segment_id)
        

1071863723 6
428403354 86
428390707 3
1071723605 53
1071723800 3
428389808 207
428390704 68
1071845849 1
428399460 6
1071846041 49
1071723711 145
1071832339 44
1071707121 4
1071707139 3
1071707171 7
226992808 3
1071846229 8
1071832389 4
226910815 2
1071754109 14
226944388 6
1071727508 5
428403353 1
1071723838 3
428389804 14
1071727490 6
1071703530 14
428403369 12
1071723686 5
1071703593 5
428402118 3
1071754197 1
1071863848 3
1071863900 3
374870394 3
1071727524 5
1071723815 3
1071729662 6
1071723637 2
1071727542 3
375114290 2
1071723676 3
1071703625 8
428390708 1
428397748 15
375160804 6
226736042 5
1071860329 2
1071845991 1
1071845973 3
428403366 4
1071754891 4
1071727530 43
1071845832 4
1071790317 8
1071846073 4
1071863801 2
374986318 1
1071845863 3
1071891776 4
1071754151 1
1071729262 1
1071832566 5
1071729215 1
428389805 3
1071729231 1
1071863817 3
428390709 3
428390706 4
428397715 1
1071863641 2
428403355 3
1071863659 3
428403373 1
1071860537 1
1071846005 2
1071845882 3
1071790234

We can see that segment_id: 428389808 has the most events

In [40]:
temp = temp[temp['SegmentID'] == 428389808]

In [41]:
temp.head(100)

,Unnamed: 0,Skapad_År,Skapad_Månad,NAME,EVENT_CODE,EVENT_TEXT,Location_text,Väg,Riktning,LANE_MASK,CREATION_TIME,COMPLETION_TIME,START_TIME,END_TIME,Note,X,Y,latitude,longitude,SegmentID
6,6,2019,Januari,2019-01-02/11:14:48-1/4,35,Olycka,E4 från Trafikplats Tönnebro (203) till Trafik...,E4,Norr,OOOOOOOOO|XOOOOOOOO,2019-01-02 11:14:48,2019-01-02 11:58:25,2019-01-02 11:14:28,2019-01-02 12:15:00,Singelolycka strax norr Tönnebro,6771941,606024,61.067895,16.964691,428389808
11,11,2019,Januari,2019-01-05/17:07:51-1/4,32,Stillastående fordon,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-05 17:07:50,2019-01-05 17:30:31,2019-01-05 17:07:51,2019-01-05 17:30:00,Personbil med haveri,6771941,606024,61.067895,16.964691,428389808
21,21,2019,Januari,2019-01-08/22:35:04-1/4,255,Bärgning,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2019-01-08 22:35:04,2019-01-08 23:07:08,2019-01-08 22:35:04,2019-01-08 23:05:00,Personbil som ska bärgas mellan Tönnebro och A...,6771941,606024,61.067895,16.964691,428389808
27,27,2019,Januari,2019-01-09/16:03:55-1/4,32,Stillastående fordon,E4 från Trafikplats Tönnebro (203) till Trafik...,E4,Norr,OOOOOOOOO|XOOOOOOOO,2019-01-09 16:03:54,2019-01-09 18:02:28,2019-01-09 16:03:55,2019-01-09 18:00:00,En bil står illa till i höjd med Tönnebro.,6771941,606024,61.067895,16.964691,428389808
32,32,2019,Januari,2019-01-12/18:13:35-1/4,32,Stillastående fordon,E4 från Trafikplats Tönnebro (203) till Trafik...,E4,Norr,OOOOOOOOO|XOOOOOOOO,2019-01-12 18:13:34,2019-01-12 19:18:45,2019-01-12 18:13:35,2019-01-12 19:30:00,Lastbil med haveri ca 1 mil norr om Tönnebro. ...,6771941,606024,61.067895,16.964691,428389808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,626,2020,Februari,2020-02-23/13:20:26-1/4,32,Stillastående fordon,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2020-02-23 13:20:26,2020-02-23 13:50:51,2020-02-23 13:20:26,2020-02-23 14:00:00,Buss med haveri strax söder om Tönnebro. Risk ...,6771941,606024,61.067895,16.964691,428389808
628,628,2020,Februari,2020-02-26/00:11:07-1/4,384,Utsläpp på vägen,E4 från Trafikplats Tönnebro (203) till Trafik...,E4,Norr,OOOOOOOOO|XOOOOOOOO,2020-02-26 00:11:06,2020-02-26 01:12:47,2020-02-26 00:10:46,2020-02-26 01:30:00,Dieselutsläpp på vägbanan.,6771941,606024,61.067895,16.964691,428389808
631,631,2020,Februari,2020-02-27/12:56:21-1/4,35,Olycka,E4 från Trafikplats Tönnebro (203) till Axmart...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2020-02-27 12:56:20,2020-02-27 14:10:34,2020-02-27 12:55:47,2020-02-27 14:10:00,Trafikolycka personbil på E4 strax söder om Tö...,6771941,606024,61.067895,16.964691,428389808
633,633,2020,Februari,2020-02-29/18:18:13-1/4,169,Långsam kö,E4 från Trafikplats Tönnebro (203) till Trafik...,E4,Syd,OOOOOOOOX|OOOOOOOOO,2020-02-29 18:18:13,2020-02-29 20:07:41,2020-02-29 18:18:13,2020-02-29 21:00:00,Långsam kö i södergående körfält på grund av t...,6771941,606024,61.067895,16.964691,428389808


In [42]:
temp.to_csv("RoadEvents_SegmentID_428389808.csv", sep=",")